In [ ]:
import numpy as np

In [ ]:
class Variable:
    def __init__(self, data):
        self.data = data

In [ ]:
from abc import ABCMeta, abstractmethod


class Function(metaclass=ABCMeta):
    def __call__(self, input, *args, **kwargs):
        x = input.data
        y = self.forward(x)
        return Variable(y)

    @abstractmethod
    def forward(self, x):
        pass

class Square(Function):
    def forward(self, x):
        return x ** 2

In [ ]:
x = Variable(np.array(10))